<a href="https://colab.research.google.com/github/vimigueloli/ICPD/blob/main/Rede_neural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A seguir será desenvolvida a Logica por trás da IA de analise e em seguida essa logica sera convertida em uma classe para implementação do codigo em qualquer aplicação

lembrando que para o funcionamento dessa classe apresentada no final do notebook sempre será necessário ajustar a licença para download de arquivos do kaggle

In [ ]:
 ##configurando o funcionamento do kaggle para a importação dos dados
import os
os.environ['KAGGLE_CONFIG_DIR']='/content'
!kaggle datasets download -d fedesoriano/stroke-prediction-dataset ##vale a pena ressaltar que para o funcionamento desse bloco é necessário upar para a aba de arquivos uma licensa de download do site kaggle

##Extraindo os arquivos que vieram compactados do kaggle
import zipfile
with zipfile.ZipFile('/content/stroke-prediction-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')

In [ ]:
##passando os dados que vão ser analizados para uma variavel
import pandas as pd
dados= pd.read_csv('healthcare-dataset-stroke-data.csv')

In [ ]:
##pre processamento dos dados transformando palavras em numeros e preenchendo valores vazios para indicar uma classificação pois IA não lida bem com strings
from sklearn.preprocessing import LabelEncoder

##preenchendo valores vazios
dados['smoking_status'] = dados['smoking_status'].replace(['Unknown'], 'never smoked')
dados[['bmi']]=dados[['bmi']].fillna(26)

##transformando strings em numeros
classificador = LabelEncoder() 
dados[['gender']] = classificador.fit_transform(dados[['gender']])
dados[['ever_married']] = classificador.fit_transform(dados[['ever_married']])
dados[['work_type']] = classificador.fit_transform(dados[['work_type']])
dados[['Residence_type']] = classificador.fit_transform(dados[['Residence_type']])
dados[['smoking_status']] = classificador.fit_transform(dados[['smoking_status']])

In [ ]:
## gerando graficos de comparação de dados do banco
import seaborn as snb

snb.pairplot(dados,hue = 'stroke')

vale apena destacar que os proximos 3 blocos de código são escolhas diferentes de trabalhar com o banco de dado e foi decidido expor as 3 opções para mostrar o estudo feito na escolha da melhor forma de lidar com os dados

In [ ]:
##aleatoriza os dados para demonstrar a rede neural no caso de que o undersampling ou o oversampling não sejam feitos
import sklearn.utils

sklearn.utils.shuffle(dados)
x= dados[['gender','age','hypertension','heart_disease','ever_married','work_type','Residence_type','avg_glucose_level','bmi','smoking_status']]
y= dados[['stroke']]

In [ ]:
##faz o undersampling
import numpy as np

## separa os pacientes que não tiveram derrame
saudaveis = dados[dados.stroke == 0].index

amostra_s = np.random.choice(saudaveis, 250, replace=False)
## separa os pacientes que tiveram derrame
doentes = dados[dados.stroke == 1].index

##junta os pacientes e sapara os dados a serem analizados das conclusões laboratoriais
amostra = dados.loc[amostra_s]
amostra = amostra.append(dados.loc[doentes])
dados = amostra
x= dados[['gender','age','hypertension','heart_disease','ever_married','work_type','Residence_type','avg_glucose_level','bmi','smoking_status']]
y= dados[['stroke']]

In [ ]:
##faz o Oversampling para lidar com o banco de dados desbalanceado
from imblearn.over_sampling import SMOTENC

##separa os dados a serem analizados das conclusões laboratoriais
x= dados[['gender','age','hypertension','heart_disease','ever_married','work_type','Residence_type','avg_glucose_level','bmi','smoking_status']]
y= dados[['stroke']]

##gera os dados de maneira estatistica
smt=SMOTENC(categorical_features=[0, 2], random_state=0)
x,y = smt.fit_resample(x,y)

## arredonda e reagrupa os dados em uma dataframe pandas 
x = pd.DataFrame(x,columns=['gender','age','hypertension','heart_disease','ever_married','work_type','Residence_type','avg_glucose_level','bmi','smoking_status'])
x['work_type'] = x['work_type'].round(0)      
x['smoking_status'] = x['smoking_status'].round(0)  
x['Residence_type'] = x['Residence_type'].round(0)
x['heart_disease'] = x['heart_disease'].round(0)
x['ever_married'] = x['ever_married'].round(0)
x['age'] = x['age'].round(0)
y = pd.DataFrame(y,columns=['stroke'])

## reconstroi o dataframe original após o oversampling para analises
dados = x
dados.insert(10, 'stroke', 0, allow_duplicates=False)
dados[['stroke']] = y

## devido a um bug na reconstrução dos dados o x é novamente separado 
x = x[['gender','age','hypertension','heart_disease','ever_married','work_type','Residence_type','avg_glucose_level','bmi','smoking_status']]


In [ ]:
##mostra os dados após o oversampling  ou under sampling
import seaborn as snb

snb.pairplot(dados,hue = 'stroke')

In [ ]:
## rede neural para classificação com porcentagem
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

## separa a amostra de teste e de treino e logo em seguida treina a rede neural
treino_x, teste_x, treino_y, teste_y = train_test_split(x,y)
modelo = SVC(probability = True, kernel= 'poly',degree=2)
modelo.fit(treino_x,treino_y)

## apos o treino testa a rede e gera alguns dados para a analise de seu desempenho e para geração de graficos
respostas_proba = (modelo.predict_proba(teste_x))*100
respostas = modelo.predict(teste_x)
acuracia = accuracy_score(teste_y,respostas)
acuracia = acuracia*100
respostas_proba =pd.DataFrame(respostas_proba,columns=['falha','sucesso']) 
respostas_proba = respostas_proba[['sucesso']]
respostas_certas = teste_y*100
print(classification_report(teste_y,respostas))

In [ ]:
## gerando um grafico para vizualização do desempenho da rede neural
import matplotlib.pyplot as mp
import numpy as np

## define o tamanho da amostra que vai para o grafico
respostas_certas = respostas_certas.head(51)
respostas_proba = respostas_proba.head(51)

## define os dados do grafico e transforma em numpy arrays para que possam ser plotados
altura1 = []
altura2 = []
altura3 = []
for eixo in respostas_certas['stroke']:
  altura1.append(eixo)
for eixo in respostas_proba['sucesso']:
  altura2.append(eixo)
for altura in altura1:
  altura3.append((altura-100)*-1)
tamanho = np.arange(51)

##plota os dados no grafico
fig,ax = mp.subplots(figsize=(20,12))
mp.bar(tamanho,altura3,color = '#92e698')
mp.bar(tamanho,altura1,color = '#e69292')
mp.plot(tamanho,respostas_proba[['sucesso']],lw=1,marker='o',ms=6,c='#4946c5')
mp.xlabel('Pacientes')
mp.ylabel('porcentagem')
mp.grid()
mp.show()

In [ ]:
##testa o paciente com a rede gerada

##recebe os dados do paciente
paciente = [[1,21,0,0,0,1,1,80,26.7,2]]
paciente = pd.DataFrame(paciente,columns=['gender','age','hypertension','heart_disease','ever_married','work_type','Residence_type','avg_glucose_level','bmi','smoking_status'])

##analiza os dados com a rede neural e mostra o resultado
chute = modelo.predict_proba(paciente) * 100
chute = pd.DataFrame(chute,columns=['falha','sucesso']) 
chute[['sucesso']]

Transformando esse cogigo em uma classe para que possa ser implementado em uma aplicação

In [ ]:
import os
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as mp
from imblearn.over_sampling import SMOTENC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score 
from sklearn.svm import SVC

class ICPD:

  ##declaração da classe
  def __init__(self,amostra=True,kernel=1):
    self.amostra = amostra
    self.kernel = kernel
    self.banco
    self.X
    self.Y
    if kernel==1:
      self.modelo = SVC(probability = True, kernel= 'rbf',degree=2)
    elif kernel==2:
      self.modelo = SVC(probability = True, kernel= 'linear',degree=2)
    elif kernel==3:
      self.modelo = SVC(probability = True, kernel= 'sigmoid',degree=2)
    elif kernel==4:
      self.modelo = SVC(probability = True, kernel= 'poly',degree=2) 
    self.certas       
    self.proba

  ##metodo que passa o banco de dados para uma variavel pandas e pre processa eles
  def iniciar(self):
    os.environ['KAGGLE_CONFIG_DIR']='/content'
    !kaggle datasets download -d fedesoriano/stroke-prediction-dataset
    with zipfile.ZipFile('/content/stroke-prediction-dataset.zip', 'r') as zip_ref:
      zip_ref.extractall('/content')
    dados= pd.read_csv('healthcare-dataset-stroke-data.csv')
    dados['smoking_status'] = dados['smoking_status'].replace(['Unknown'], 'never smoked')
    dados[['bmi']]=dados[['bmi']].fillna(26)
    classificador = LabelEncoder() 
    dados[['gender']] = classificador.fit_transform(dados[['gender']])
    dados[['ever_married']] = classificador.fit_transform(dados[['ever_married']])
    dados[['work_type']] = classificador.fit_transform(dados[['work_type']])
    dados[['Residence_type']] = classificador.fit_transform(dados[['Residence_type']])
    dados[['smoking_status']] = classificador.fit_transform(dados[['smoking_status']])
    self.banco = dados

  ##metodo que faz o ajuste da amostra via overampling caso 'amostra' == FALSE ou undersampling caso 'amostra' == TRUE  
  def ajuste(self):
    dados = self.banco
    if self.amostra == True:
      saudaveis = dados[dados.stroke == 0].index
      amostra_s = np.random.choice(saudaveis, 250, replace=False)
      doentes = dados[dados.stroke == 1].index
      amostra = dados.loc[amostra_s]
      amostra = amostra.append(dados.loc[doentes])
      dados = amostra
      x= dados[['gender','age','hypertension','heart_disease','ever_married','work_type','Residence_type','avg_glucose_level','bmi','smoking_status']]
      y= dados[['stroke']]
    else:
      x= dados[['gender','age','hypertension','heart_disease','ever_married','work_type','Residence_type','avg_glucose_level','bmi','smoking_status']]
      y= dados[['stroke']]
      smt=SMOTENC(categorical_features=[0, 2], random_state=0)
      x,y = smt.fit_resample(x,y)
      x = pd.DataFrame(x,columns=['gender','age','hypertension','heart_disease','ever_married','work_type','Residence_type','avg_glucose_level','bmi','smoking_status'])
      x['work_type'] = x['work_type'].round(0)      
      x['smoking_status'] = x['smoking_status'].round(0)  
      x['Residence_type'] = x['Residence_type'].round(0)
      x['heart_disease'] = x['heart_disease'].round(0)
      x['ever_married'] = x['ever_married'].round(0)
      x['age'] = x['age'].round(0)
      y = pd.DataFrame(y,columns=['stroke'])
      dados = x
      dados.insert(10, 'stroke', 0, allow_duplicates=False)
      dados[['stroke']] = y
      x = x[['gender','age','hypertension','heart_disease','ever_married','work_type','Residence_type','avg_glucose_level','bmi','smoking_status']]
    self.banco= dados
    self.X= x
    self.Y= y 

  ##metodo que treina a IA e armazena dados para a geração do grafico
  def treino(self):
    x= self.X
    y= self.Y 
    treino_x, teste_x, treino_y, teste_y = train_test_split(x,y)
    self.modelo.fit(treino_x,treino_y)
    respostas_proba = (self.modelo.predict_proba(teste_x))*100
    respostas = modelo.predict(teste_x)
    respostas_proba =pd.DataFrame(respostas_proba,columns=['falha','sucesso']) 
    respostas_proba = respostas_proba[['sucesso']]
    respostas_certas = teste_y*100
    self.certas= respostas_certas
    self.proba = respostas_proba

  ##metodo que gera um grafico para exibição
  def grafico(self):
    respostas_certas = self.certas.head(51)
    respostas_proba = self.proba.head(51)
    altura1 = []
    altura2 = []
    altura3 = []
    for eixo in respostas_certas['stroke']:
      altura1.append(eixo)
    for eixo in respostas_proba['sucesso']:
      altura2.append(eixo)
    for altura in altura1:
      altura3.append((altura-100)*-1)
    tamanho = np.arange(51)
    fig,ax = mp.subplots(figsize=(20,12))
    mp.bar(tamanho,altura3,color = '#92e698')
    mp.bar(tamanho,altura1,color = '#e69292')
    mp.plot(tamanho,respostas_proba[['sucesso']],lw=1,marker='o',ms=6,c='#4946c5')
    mp.xlabel('Pacientes')
    mp.ylabel('porcentagem')
    mp.grid()
    mp.show()
    mp.savefig("desempenho.png",dpi=300)
  
  ##metodo que recebe os dados do paciente e da um valor em porcentagem da chance dessa pessoa ter derrame de acordo com o banco de dados analizados
  def diagnostico(self, genero, age, hypertension, heart_disease, ever_married, work_type, Residence_type, avg_glucose_level, bmi, smoking_status):
    paciente = [[genero, age, hypertension, heart_disease, ever_married, work_type, Residence_type, avg_glucose_level, bmi, smoking_status]]
    paciente = pd.DataFrame(paciente,columns=['gender','age','hypertension','heart_disease','ever_married','work_type','Residence_type','avg_glucose_level','bmi','smoking_status'])
    chute = self.modelo.predict_proba(paciente) * 100
    chute = pd.DataFrame(chute,columns=['falha','sucesso']) 
    return chute[['sucesso']]